In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import seaborn as sns
import sklearn.cross_validation as cv
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor,export_graphviz
from DecisionTree import DecisionTree
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import GradientBoostingRegressor as GBR
from GradientBoosting import GradientBoosting as myGB
# Plotting config
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/popka/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
FOLDER = "data/"
FILES = [
        "iris.txt", "bezdekIris.txt", "wine.txt", "bupa.txt", "housing.txt", "auto-mpg.txt", "spam"
        ]
FILE = "spam"

In [3]:
# Подготавливаем признаки и целевую функцию
if FILE in FILES[:6]:
    
    df = pd.read_csv(FOLDER+FILE, sep=",", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    
    if FILE in FILES[:2]:
        # ИРИСЫ
        df[4] = pd.factorize(df[4])[0]
        X = df[[0,1,2,3]].as_matrix()
        y = df[4].as_matrix()

    if FILE == FILES[2]:
        x_indexes = [x for x in range(1,14)]
        X = df[x_indexes].as_matrix()
        y = df[0].as_matrix()

    if FILE == FILES[3]:
        X = df[[0,1,2,3,4,5]].as_matrix()
        y = df[6]
        
    if FILE == FILES[4]:
        df = pd.read_csv(FOLDER+FILE, sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
        X = df[df.columns[1:]].as_matrix()
        y = df[df.columns[0]].as_matrix()
        
    if FILE == FILES[5]:
        df = pd.read_csv(FOLDER+FILE, sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
        X = df[df.columns[1:-1]].as_matrix()
        y = df[df.columns[0]].as_matrix()
        
    x_train, x_test, y_train, y_test = cv.train_test_split(X, y, test_size=0.25)

else:
    
    df_train = pd.read_csv(FOLDER+FILE+".train.txt", sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    df_test = pd.read_csv(FOLDER+FILE+".test.txt", sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    x_train = df_train[df_train.columns[1:]].as_matrix()
    y_train = df_train[df_train.columns[0]].as_matrix()
    x_test = df_test[df_test.columns[1:]].as_matrix()
    y_test = df_test[df_test.columns[0]].as_matrix()

In [4]:
def plot_curve(x, y, title):
    
    fig = plt.figure(figsize=(10, 10))
    plt.plot(x, y, color='blue', label='MyGB')
    plt.ylim(0, max(y*1.03)+0.02)
    plt.grid(True)
    plt.xlabel("features count", fontsize=15)
    plt.ylabel("mse", fontsize=15)
    plt.title(title)
    plt.legend()
    #plt.show()
    return

## Сколько фичей отбираем 

## Embedded 

In [5]:
N = 40

In [6]:
n_est=250
max_depth=4
rsm=False
shrinkage=0.1

In [7]:
%%time
my_gb = myGB(n_estimators=n_est, max_depth=max_depth, rsm=rsm, shrinkage=shrinkage)
my_gb.fit(x_train, y_train)
y_predict = my_gb.predict(x_test)
print mse(y_test, y_predict)

0.065097389117
CPU times: user 20min 33s, sys: 2.59 s, total: 20min 36s
Wall time: 20min 40s


/home/popka/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/popka/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [8]:
def aggregate_gains(node, feature_importance):
    if node.is_leaf:
        return
    
    gain = node.predicate.gain
    feature_index = node.predicate.feature_id
    feature_importance[feature_index] = gain
    
    aggregate_gains(node.left_node, feature_importance)
    aggregate_gains(node.right_node, feature_importance)

In [9]:
feature_importance = np.zeros(len(x_train.T))
ensamble = my_gb._estimators
for tree in ensamble:
    aggregate_gains(tree._root, feature_importance)

In [10]:
zipped = zip(feature_importance, df_train.columns[1:])
zipped.sort(key = lambda t: t[0], reverse=True)
zipped[:N]

[(0.047494489699602127, 95),
 (0.042150773108005524, 6),
 (0.033429037779569626, 19),
 (0.028516422957181931, 4),
 (0.026696523651480675, 7),
 (0.024460051208734512, 71),
 (0.018712038174271584, 64),
 (0.018295813351869583, 30),
 (0.016621118411421776, 15),
 (0.016550984233617783, 3),
 (0.015333914197981358, 31),
 (0.015091545879840851, 34),
 (0.014703515917062759, 53),
 (0.013742998242378235, 58),
 (0.012283220887184143, 59),
 (0.010555494576692581, 8),
 (0.0093645807355642319, 41),
 (0.0082793775945901871, 44),
 (0.0081197386607527733, 57),
 (0.0080301119014620781, 102),
 (0.0080185551196336746, 1),
 (0.0069031361490488052, 39),
 (0.00582143384963274, 40),
 (0.0049374401569366455, 24),
 (0.0049288328737020493, 22),
 (0.0046140635386109352, 18),
 (0.0039362385869026184, 38),
 (0.0037562530487775803, 43),
 (0.0033722412772476673, 21),
 (0.0031828256323933601, 9),
 (0.0030789165757596493, 92),
 (0.0030155307613313198, 2),
 (0.0027428208850324154, 26),
 (0.002675558440387249, 14),
 (0.00

In [ ]:
[(0.047494489699602127, 95),
 (0.042150773108005524, 6),
 (0.033429037779569626, 19),
 (0.028516422957181931, 4),
 (0.026696523651480675, 7),
 (0.024460051208734512, 71),
 (0.018712038174271584, 64),
 (0.018295813351869583, 30),
 (0.016621118411421776, 15),
 (0.016550984233617783, 3),
 (0.015333914197981358, 31),
 (0.015091545879840851, 34),
 (0.014703515917062759, 53),
 (0.013742998242378235, 58),
 (0.012283220887184143, 59),
 (0.010555494576692581, 8),
 (0.0093645807355642319, 41),
 (0.0082793775945901871, 44),
 (0.0081197386607527733, 57),
 (0.0080301119014620781, 102),
 (0.0080185551196336746, 1),
 (0.0069031361490488052, 39),
 (0.00582143384963274, 40),
 (0.0049374401569366455, 24),
 (0.0049288328737020493, 22),
 (0.0046140635386109352, 18),
 (0.0039362385869026184, 38),
 (0.0037562530487775803, 43),
 (0.0033722412772476673, 21),
 (0.0031828256323933601, 9),
 (0.0030789165757596493, 92),
 (0.0030155307613313198, 2),
 (0.0027428208850324154, 26),
 (0.002675558440387249, 14),
 (0.0025639976374804974, 47),
 (0.0017780180787667632, 75),
 (0.0017490275204181671, 89),
 (0.0015673665329813957, 101),
 (0.0011820010840892792, 76),
 (0.0011515934020280838, 60)]

#### Посчитаем качество на сокращенном наборе фичей

In [19]:
good_features = []
for i in range(N):
    good_features.append(zipped[i][1])

In [ ]:
mse_array_embedded = []
for i in range(1, len(good_features)+1):
    features = good_features[:i]
    X_train, X_test = x_train[:,features], x_test[:, features]
    
    my_gb = myGB(n_estimators=n_est, max_depth=max_depth, rsm=rsm, shrinkage=shrinkage)
    my_gb.fit(X_train, y_train)
    y_predict = my_gb.predict(X_test)
    m = mse(y_test, y_predict)
    mse_array_embedded.append(m)
    print i, m

#### Сравним качество со Sklearn'ом 

In [ ]:
gb = GradientBoostingRegressor(n_estimators=250)
gb.fit(x_train, y_train)

In [ ]:
zipped = zip(gb.feature_importances_, df_train.columns[1:])
zipped.sort(key = lambda t: t[0], reverse=True)

In [ ]:
new_features = []
for i in range(N):
    new_features.append(zipped[i][1]-1)
    
new_x_train = x_train[:,new_features]

In [ ]:
gb = GradientBoostingRegressor(n_estimators=250)
gb.fit(new_x_train, y_train)
y_predict = my_gb.predict(x_test)
print mse(y_test, y_predict)

# Filtering

#### Считаем дисперсию. Удаляем фичи с малой дисперсией

In [ ]:
zipped_var = zip(np.var(x_train, axis=1), df_train.columns[1:])
zipped_var.sort(key = lambda t: t[0], reverse=False)
zipped_var[:N]

#### Считаем корреляцию

In [ ]:
corr_mat = df_train.corr().abs()
plt.figure(figsize=(18, 18))
plt.title("Correlation between features")
sns.heatmap(corr_mat, cmap="YlOrRd")

In [ ]:
# Выберем признаки, похожие на ответ
abs_corr_y = corr_mat[0]
good_features = abs_corr_y.sort_values(ascending=False)[:N]
good_features

#### Найдем признаки, которых похожи на те, что мы отобрали. Их не берем

In [ ]:
min_cor = 0.7
feature_indexes = good_features.index
full_similar_features = []
for idx in feature_indexes:
    similiar_features = corr_mat[corr_mat[idx]>min_cor].index.values
    # Удаляем значение idx из similiar_features
    similiar_features = similiar_features[similiar_features != idx]
    full_similar_features.extend(similiar_features)
    print "Feature ", idx, " correlate with ", similiar_features
    
print full_similar_features

In [ ]:
for f in full_similar_features:
    if f in good_features:
        good_features = good_features.drop(f)

In [ ]:
good_features = list(good_features.index[1:]-1)

In [ ]:
# чтобы не потерять
print good_features

In [ ]:
mse_array_filtering=[]
for i in range(1, len(good_features)+1):
    features = good_features[:i]
    X_train, X_test = x_train[:,features], x_test[:, features]
    model = GBR(n_estimators=n_est, max_depth=max_depth, learning_rate=shrinkage)
    model.fit(X_train, y_train)
    y_predicted=model.predict(X_test)
    mse_array_filtering.append(mse(y_test, y_predicted))

In [ ]:
plot_curve(range(1, len(good_features)+1), np.asarray(mse_array), "Filtering")

# Wrapper

In [ ]:
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import GradientBoostingRegressor as GBR

In [ ]:
n_est=250
rsm=False
shrinkage=0.1
max_depth=4

In [ ]:
def measure_quality(X, y, k=3):
    
    kf = KFold(len(y), n_folds=k)
    mse_array = []
    for train, test in kf:
        _X_train, _X_test, _y_train, _y_test = X[train], X[test], y[train], y[test]
        model = GBR(n_estimators=n_est, max_depth=max_depth, learning_rate=shrinkage)
        model.fit(_X_train, _y_train)
        _y_predicted=model.predict(_X_test)
        mse_array.append(mse(_y_test, _y_predicted))
    
    return float(sum(mse_array))/len(mse_array)

In [ ]:
def find_best_feature(best_features, remaining_features, x_train, y_train):
    """
    Выбираем лучшую фичу.
    best_features - [1,5,10,...]фичи, которые уже были выбраны
    remaining_features - [2,3,4,6,7,8,9,,...] оставшиеся фичи, которые уже были выбраны
    """
    best_mse = sys.maxint
    best_fea = None
    print best_features, remaining_features

    for f in remaining_features:
        new_features = best_features + [f]
        print new_features
        new_mse = measure_quality(x_train[:,new_features], y_train)
        
        if new_mse < best_mse:
            best_mse = new_mse
            best_fea = f
            
    return best_fea, best_mse

In [ ]:
%%time
best_features=[]

all_features=range(0, x_train.shape[1])
print x_train.shape[1]
best_mse=sys.maxint

for f in all_features:
    feature, cur_mse = find_best_feature(best_features, all_features, x_train, y_train)
    
    if cur_mse<best_mse:
        print "===================="
        print "new_best_mse = ", cur_mse
        best_mse=cur_mse
        best_features.append(feature)
        all_features.remove(feature)
        print "the_best_feature_list=", best_features
        print all_features
        print "////////////////"
    else:
        print "NOT_GOOD_FEATURES"
        break

In [ ]:
# Считалось 5 часов на Sklern'e, с моей реализацией не считается!

In [ ]:
#good_features = [48, 70, 53, 10, 101, 56, 55, 9, 71, 4, 76, 52, 16, 3, 6, 50, 65, 100]
good_features = [48, 70, 53, 55, 52, 74, 29, 8, 71, 83, 10, 43, 4, 54, 41, 76, 35, 14, 9, 45, 51, 75, 60, 101]

mse_array_wrapper=[]
for i in range(1, len(good_features)+1):
    features = good_features[:i]
    X_train, X_test = x_train[:,features], x_test[:, features]
    model = GBR(n_estimators=n_est, max_depth=max_depth, learning_rate=shrinkage)
    model.fit(X_train, y_train)
    y_predicted=model.predict(X_test)
    mse_array_wrapper.append(mse(y_test, y_predicted))

In [ ]:
plot_curve(range(1, len(good_features)+1), np.asarray(mse_array), "Wrapper")